<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/formulacion/formulacion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descarga de bibliotecas a utilizar

In [122]:
# Se descargan las bibliotecas necesarias
# !pip install nltk
# !pip install googletrans
# !pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.4 MB/s eta 0:00:00


In [215]:
# Se importan las bibliotecas a utilizar

# Pandas y numpy
import pandas as pd
import numpy as np

# Detección de idioma
from googletrans import Translator
translator = Translator()
from langdetect import detect, LangDetectException

#
import itertools

# Gráficación
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Análisis de lenguaje natural
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
import re

In [74]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Se cargan las bases

In [136]:
# Se traen las rutas de los archivos a trabajar.
games_ruta = "/content/games_clean.json.gz"
genres_ruta = "/content/genres_games.json.gz"
specs_ruta = "/content/specs_games.json.gz"
reviews_ruta = "/content/reviews_limpia.json.gz"
items_ruta = "/content/items_limpia.json.gz"

In [137]:
# Se convierten a pandas.

In [138]:
games = pd.read_json(games_ruta,compression="gzip")
games.head()

,name,release_date,id_game,developer
0,Lost Summoner Kitty,2018-01-04,761140,Kotoshiro
1,Ironbound,2018-01-04,643980,Secret Level SRL
2,Real Pool 3D - Poolians,2017-07-24,670290,Poolians.com
3,弹炸人2222,2017-12-07,767400,彼岸领域
4,Log Challenge,None,773570,None


In [139]:
genres = pd.read_json(genres_ruta,compression="gzip")
genres.head()

,id_game,genre
0,761140,Strategy
1,761140,Action
2,761140,Indie
3,761140,Casual
4,761140,Simulation


In [140]:
specs = pd.read_json(specs_ruta,compression="gzip")
# specs["id_game"] = specs["id_game"].astype(int)
specs.head()

,id_game,specs
0,761140,Single-player
1,643980,Single-player
2,643980,Multi-player
3,643980,Online Multi-Player
4,643980,Cross-Platform Multiplayer


In [229]:
reviews = pd.read_json(reviews_ruta,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews.head()

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08


In [142]:
items = pd.read_json(items_ruta,compression="gzip")
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5159890,76561198329548331,346330,BrainBread 2,0,0
5159891,76561198329548331,373330,All Is Dust,0,0
5159892,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5159893,76561198329548331,521570,You Have 10 Seconds 2,4,4


# Procesamiento del lenguaje natural en las reviews.

In [230]:
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


In [231]:
# Se obtiene información sobre la distribución del dataset.
reviews["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [232]:
# Se transliteran los caracteres de "review" a ASCII
reviews["review"] = reviews["review"].apply(unidecode)
# Se colocan todos los caracteres en minúscula y se asegura que sean strings.
reviews["review"] = reviews["review"].str.lower()

# deteccion del idioma

In [219]:
# Se busca obtener el lenguaje de cada una de las reviews, pero se levanta un error:
reviews["langugage"] = reviews["review"].apply(detect)

LangDetectException: ignored

In [222]:
# Se crea una función para detectar el lenguaje (ya que aplicar limpieza de manera directa no estaba dando resultado)
# En caso de que no sea capaz de detectarlo, no trae ningún valor.
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        # print(f"Error while detecting language: {e}")
        return None

reviews["language"] = reviews["review"].apply(detect_language)

In [223]:
# Se revisa la base obtenida.
reviews

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,en
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,en
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14,en
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,en
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31,en
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14,de


In [224]:
# Se revisan los idiomas obtenidos a partir de la detección.
reviews["language"].unique()

array(['en', 'ca', 'so', 'tl', 'cy', 'th', 'pl', 'de', 'pt', 'es', 'ru',
       'id', 'lt', 'da', 'af', 'it', 'no', 'sw', 'zh-cn', 'nl', None,
       'cs', 'fi', 'hu', 'sv', 'ro', 'et', 'hr', 'tr', 'lv', 'sk', 'fr',
       'sq', 'sl', 'ko', 'vi', 'bg', 'ja', 'zh-tw', 'uk', 'ta', 'mk',
       'ar'], dtype=object)

In [228]:
x = reviews[reviews["language"] == "ca"]
x

,user_id,item_id,recommend,review,review_date,language
16,maplemage,211420,True,Git gud,2014-04-15,ca
380,76561198078799606,233450,True,Very unique game and VERY addicive,2014-05-02,ca
429,76561198063624801,440,True,Is a brillant game,2014-01-12,ca
552,76561198047185525,213670,True,Classic racist fun,2014-03-20,ca
733,Barridos,4000,True,Todavia depende del css?-_-,2014-04-09,ca
...,...,...,...,...,...,...
42763,76561198102469448,4000,True,its a fun game,2014-05-28,ca
42922,lomardriver,440,True,RUIN MY LIFE LOL 5STAR,2014-04-17,ca
43041,76561198108193259,206420,True,its funny,2014-01-20,ca
43650,Ash1261,318430,True,lol 14 cents,2015-03-07,ca


In [212]:
# Se transliteran los caracteres de "review" a ASCII
reviews["review"] = reviews["review"].apply(unidecode)
# Se colocan todos los caracteres en minúscula y se asegura que sean strings.
reviews["review"] = reviews["review"].str.lower()

<ipython-input-212-b26fc4117e0e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["review"] = reviews["review"].apply(unidecode)
<ipython-input-212-b26fc4117e0e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["review"] = reviews["review"].str.lower()


In [214]:
# Se vuelve a intentar detectar el lenguaje para cada una de las reviews
reviews["langugage"] = reviews["review"].apply(detect)

LangDetectException: ignored

In [175]:
x = reviews.loc[41500,"review"]
print(type(x))
print(x)
print(detect(x))

<class 'str'>
made my singleplayer wouldn't work on gmod!
en


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43976 entries, 0 to 43975
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      43976 non-null  object        
 1   item_id      43976 non-null  int64         
 2   recommend    43976 non-null  bool          
 3   review       43976 non-null  object        
 4   review_date  43976 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 1.7+ MB
